### Step 1: Import Required Libraries

I begin by importing essential Python libraries:
- `cv2` for image processing
- `numpy` for numerical operations
- `matplotlib.pyplot` for visualization
- `os` for file handling
- `KMeans` from `sklearn.cluster` for clustering or label grouping

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans


###  Step 2: Define Dice Score Function

The Dice Score measures similarity between the predicted and ground truth masks. It ranges from 0 (no overlap) to 1 (perfect match). This function takes two binary masks and returns their Dice Score.


In [4]:
def dice_score(pred, gt):
    intersection = np.sum(pred * gt)
    return (2. * intersection) / (np.sum(pred) + np.sum(gt) + 1e-8)

def iou_score(pred, gt):
    intersection = np.sum(pred * gt)
    union = np.sum(pred) + np.sum(gt) - intersection
    return intersection / (union + 1e-8)


### Step 3: Load Image and Mask Paths

We specify the paths to:
- `images/`: folder containing predicted segmentation outputs
- `masks/`: folder with corresponding ground truth masks




In [5]:
image_folder = 'images'
mask_folder = 'masks'


###  Step 4: Evaluate Predictions Using Dice and IoU

This loop reads each predicted image and its corresponding ground truth mask. It:
- Resizes them to match
- Thresholds to convert to binary masks
- Calculates Dice Score and Intersection over Union (IoU)
- Stores scores in a list for summary

Finally, it prints the average Dice and IoU scores for all images.


In [6]:
dice_scores = []
iou_scores = []
for i in range(1, 197):  # 196 images
    print(f"Processing image {i}...")
    image_path = f'{image_folder}/{i}.tif'
    mask_path = f'{mask_folder}/p{i}.tif'
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        continue
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))  
    mask = cv2.imread(mask_path, 0)
    mask = cv2.resize(mask, (256, 256))
    mask = (mask > 127).astype(np.uint8)
    segmented_img, labels_reshaped = kmeans_segmentation(image, k=3)
    best_dice = 0
    best_iou = 0
    best_mask = None
    for cluster in range(3):
        candidate_mask = (labels_reshaped == cluster).astype(np.uint8)
        kernel = np.ones((3,3), np.uint8)
        candidate_mask = cv2.morphologyEx(candidate_mask, cv2.MORPH_OPEN, kernel)
        candidate_mask = cv2.morphologyEx(candidate_mask, cv2.MORPH_CLOSE, kernel)
        dice = dice_score(candidate_mask, mask)
        iou = iou_score(candidate_mask, mask)
        if dice > best_dice:
            best_dice = dice
            best_iou = iou
            best_mask = candidate_mask
    dice_scores.append(best_dice)
    iou_scores.append(best_iou)
    print(f"Dice Score: {best_dice:.4f}, IoU: {best_iou:.4f}")
    if i <= 10:  
        plt.figure(figsize=(12,4))
        plt.subplot(1, 4, 1)
        plt.imshow(image)
        plt.title("Original Image")
        plt.axis('off')
        plt.subplot(1, 4, 2)
        plt.imshow(segmented_img)
        plt.title("K-Means Output")
        plt.axis('off')
        plt.subplot(1, 4, 3)
        plt.imshow(best_mask, cmap='gray')
        plt.title("Predicted Mask")
        plt.axis('off')
        plt.subplot(1, 4, 4)
        plt.imshow(mask, cmap='gray')
        plt.title("Ground Truth Mask")
        plt.axis('off')
        plt.show()
if dice_scores and iou_scores:
    avg_dice = np.mean(dice_scores)
    avg_iou = np.mean(iou_scores)
    print(f"\nAverage Dice Score: {avg_dice:.4f}")
    print(f"Average IoU Score: {avg_iou:.4f}")
    if avg_dice > 0.7 and avg_iou > 0.7:
        print("Success: Both average Dice and IoU are above 0.7!")
    else:
        print("Warning: Average Dice or IoU is below 0.7. Consider tuning your segmentation.")
else:
    print("No images were processed.")

Processing image 1...
Image file not found: images/1.tif
Processing image 2...
Image file not found: images/2.tif
Processing image 3...
Image file not found: images/3.tif
Processing image 4...
Image file not found: images/4.tif
Processing image 5...
Image file not found: images/5.tif
Processing image 6...
Image file not found: images/6.tif
Processing image 7...
Image file not found: images/7.tif
Processing image 8...
Image file not found: images/8.tif
Processing image 9...
Image file not found: images/9.tif
Processing image 10...
Image file not found: images/10.tif
Processing image 11...
Image file not found: images/11.tif
Processing image 12...
Image file not found: images/12.tif
Processing image 13...
Image file not found: images/13.tif
Processing image 14...
Image file not found: images/14.tif
Processing image 15...
Image file not found: images/15.tif
Processing image 16...
Image file not found: images/16.tif
Processing image 17...
Image file not found: images/17.tif
Processing imag